In [ ]:
pip install sentence-transformers torch faiss-cpu

In [24]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import math
from tqdm import tqdm
import faiss
import numpy as np

In [14]:
wine = pd.read_csv('winemag-data-130k-v2.csv', encoding='utf-8', on_bad_lines='skip')
wine.rename(columns={'Unnamed: 0': 'id'}, inplace=True)


In [15]:
wine.head()

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Generating and Exporting Description Embeddings to a Tensor File

In [ ]:
# This code was used to create the description embeddings tensor file.
# You don't have to run it again.
# It takes about an hour to run.

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Extract the "description" column as a list
descriptions = wine['description'].tolist()

# Set batch size
batch_size = 100

# Calculate the number of batches needed
num_batches = math.ceil(len(descriptions) / batch_size)

# Initialize a list to store embeddings
description_embeddings = []

# Process data in batches with tqdm
for i in tqdm(range(num_batches), desc="Processing Batches"):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(descriptions))
    batch_descriptions = descriptions[start_idx:end_idx]

    # Encode the batch of descriptions using the model
    batch_embeddings = model.encode(batch_descriptions)

    # Append the batch embeddings to the list
    description_embeddings.extend(batch_embeddings)

# Convert the list of embeddings to a PyTorch tensor
description_tensor = torch.tensor(description_embeddings)

# Save the tensor to a file
torch.save(description_tensor, 'description_embeddings.pt')


Processing Batches: 100%|██████████| 1300/1300 [57:37<00:00,  2.66s/it]
<ipython-input-20-269cb6eaabd0>:35: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  description_tensor = torch.tensor(description_embeddings)


# KNN Search with Faiss

In [16]:
knn_vectors = torch.load("description_embeddings.pt")
knn_labels = wine.id.astype(str).tolist()

In [35]:
# Convert the data to NumPy arrays for use with faiss
vectors_np = knn_vectors.numpy()

# Determine the dimension of the vectors
dimension = vectors_np.shape[1]  # This represents the dimension of the vectors

# Build the Faiss index
index = faiss.IndexFlatL2(dimension)  # Create a Faiss index with L2 (Euclidean) distance metric
index.add(vectors_np)  # Add the data vectors to the index


In [40]:
# Initialize a sentence transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode the text query to obtain a vector
query_text = 'melon and bitter'
query_vector = model.encode([query_text])[0]

# Query the index with the vector
k = 20  # Number of neighbors to return

# Perform a nearest neighbor search to find the closest neighbors to the query vector(s).
# D will contain squared L2 distances between the query vector and its neighbors.
# I will contain the indices of the nearest neighbors in the dataset.
D, I = index.search(np.array([query_vector]), k)

# Get labels of the neighbors
neighbor_labels = [knn_labels[i] for i in I[0]]

# Extract rows from the wine DataFrame
result = wine.iloc[I[0]]




In [41]:
result['description'].values

array(['A positive bouquet of melon, white flowers and honey leads to a ripped acidic palate that generates tangy, almost sharp flavors of orange and gooseberry. A citrusy angular finish turns pithy and bitter as it dissipates. Drink now.',
       'Melon and green fruit aromas have a pinch of almond bitterness. This feels fresh but low in volume, with standard apple and green melon flavors. It finishes soft, with notes of mustard seed and fennel.',
       'Mellow and melony up front, with a plump, bland palate featuring mealy melon and peach flavors. The finish has a touch of bitterness.',
       'Largely neutral aromas touch on melon, peach and toast. This is smooth and melony on the palate, with flavors of bitter nut skins, melon, banana and papaya. A touch of bitterness rises up on the finish.',
       'Melony, tropical aromas are clean and holding steady. This is on the full and oily side, with citric acidity creating a centerweight. Punchy citrus, melon and peach flavors finish wi